In [1]:
import sys
import os

# 将当前notebook所在目录（即本文件的上层目录）加入到sys.path中
notebook_dir = "/Users/muwang/Documents/github/TomoPANDA-pick/"
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)






In [2]:
from utils.io_dynamo import dynamo_tbl_vll_to_relion_star

In [9]:
project_dir = '/Users/muwang/Documents/github/TomoPANDA-pick/'
vll_path = os.path.join(project_dir, 'notebooks/input/dcp_single_030/crop.vll')
tbl_path = os.path.join(project_dir, 'notebooks/input/dcp_single_030/refined_table_ref_001_ite_0004.tbl')
rl_particles_path = os.path.join(project_dir, 'notebooks/dcp_single_030/refined_table_ref_001_ite_0004.star')

In [10]:
dynamo_tbl_vll_to_relion_star(tbl_path, vll_path, output_file=rl_particles_path,pixel_size=6.74,tomogram_size=(999,999,499),output_centered=True)

Using rlnMicrographName column
Converted 21038 particles from Dynamo format to RELION format.
Output written to: /Users/muwang/Documents/github/TomoPANDA-pick/notebooks/dcp_single_030/refined_table_ref_001_ite_0004.star


,rlnCenteredCoordinateXAngst,rlnCenteredCoordinateYAngst,rlnCenteredCoordinateZAngst,rlnAngleRot,rlnAngleTilt,rlnAnglePsi,rlnOriginXAngst,rlnOriginYAngst,rlnOriginZAngst,rlnMicrographName
0,-1604.12,-2992.56,256.12,-133.444,70.291,102.0480,-5.034713,106.727900,-15.990650,rec_MGS5_001_T1_ts_001
1,2844.28,-1482.80,-1091.88,-8.762,115.980,-73.6700,61.389268,-18.870652,48.019130,rec_MGS5_001_T1_ts_001
2,-1550.20,-2952.12,94.36,-133.398,63.649,107.8360,-62.667846,93.012000,107.590620,rec_MGS5_001_T1_ts_001
3,2817.32,-2332.04,-1267.12,-11.767,127.390,74.6970,-94.737440,129.670860,-80.327320,rec_MGS5_001_T1_ts_001
4,2857.76,-1132.32,-1011.00,-23.314,121.530,-113.3500,1.281004,-98.444440,-41.859444,rec_MGS5_001_T1_ts_001
...,...,...,...,...,...,...,...,...,...,...
21033,188.72,-2453.36,-849.24,117.010,114.590,-138.8000,-87.997440,-54.744302,6.863342,rec_pinkss_MGS002_T2_ts_010
21034,-984.04,-2763.40,889.68,3.008,89.142,-99.8200,-100.095740,-87.512160,10.880382,rec_pinkss_MGS002_T2_ts_010
21035,593.12,-1523.24,-1172.76,-75.336,80.084,-96.0400,-54.182186,-80.232960,102.845660,rec_pinkss_MGS002_T2_ts_010
21036,2412.92,-2642.08,-835.76,-47.718,89.167,82.5553,-139.922400,66.670058,192.615720,rec_pinkss_MGS002_T2_ts_010


In [11]:
# read the star file by starfile
import starfile
rl_particles = starfile.read(rl_particles_path)
# add a new column 'rlnTomoName' to the star file, set the value from 1 based on the rlnMicrographName column. 不同的MicrographName顺次编号
# 利用每个不同的 rlnMicrographName 生成不同的序号，从1开始
import pandas as pd

# 获取唯一的MicrographName，并建立映射关系
unique_micrographs = rl_particles['rlnMicrographName'].unique()
micrograph_to_tomo_number = {name: idx+1 for idx, name in enumerate(unique_micrographs)}

# 根据映射为每行赋予编号
rl_particles['rlnTomoName'] = rl_particles['rlnMicrographName'].map(micrograph_to_tomo_number)

# write the star file
rl_particles_path = rl_particles_path.replace('.star', '_tomo_name.star')
starfile.write(rl_particles, rl_particles_path)


In [19]:
rl_particles_path ="/Volumes/SSD_1t_001/tomograms/merged_particles.star"
rl_particles = starfile.read(rl_particles_path)
unique_micrographs = rl_particles['rlnTomoName'].unique()
micrograph_to_tomo_number = {name: idx+1 for idx, name in enumerate(unique_micrographs)}
rl_particles['rlnMicrographName'] = rl_particles['rlnTomoName']
rl_particles['rlnTomoName'] = rl_particles['rlnMicrographName'].map(micrograph_to_tomo_number)
rl_particles_path = rl_particles_path.replace('.star', '_tomo_name.star')
starfile.write(rl_particles, rl_particles_path)


In [2]:
from utils.io_dynamo import relion_star_to_dynamo_tbl

starfile_particle = "/Users/muwang/Documents/github/TomoPANDA-pick/notebooks/dcp_single_030/refined_table_ref_001_ite_0004_tomo_name.star"

# convert to dynamo tbl vll
# check pixel size, tomogram size first
# will save to output/particles_changeTomoName_bin8.tbl
df =relion_star_to_dynamo_tbl(
    starfile_particle, 
    6.72, # 现在 tomo 的 pixel size
    tomogram_size=(999,999,499),
    bin_scalar=8.0, # 现在的tomogram_size是bin8的，所以需要乘以8
    output_file=starfile_particle.replace('.star', '_bin8.tbl')
)

In [ ]:
from utils.io_dynamo import read_vll_to_df, save_sorted_vll_by_tomonames
# extract tomogram names from star file
df_star = starfile.read(starfile_particle)
tomo_names = df_star['rlnTomoName'].unique()

# re-orgnize vll file by tomogram names and save to output/sorted_tomos_bin8.vll
#vll_path = '/home/muwang/Documents/projects/IR/20230518IR_mCherry/all_tomograms_bin8.txt'
#vll_df = read_vll_to_df(vll_path)
#save_sorted_vll_by_tomonames(tomo_names, vll_df, "output/sorted_tomos_bin8.vll")